In [15]:
import pandas as pd
import json
from tqdm import tqdm


In [34]:
from openai.embeddings_utils import get_embedding
engine = 'text-embedding-ada-002'
len(get_embedding('text', engine=engine))

1536

In [3]:
df_exist = pd.concat([
    pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_train_oaiemb.json', lines=True),
    pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_val_oaiemb.json', lines=True),
    pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_test_oaiemb.json', lines=True)
])

In [5]:
df_train = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_query_attr_train.json', 
    lines=True)
df_val = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_query_attr_val.json', 
    lines=True)
df_test = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_query_attr_test.json', 
    lines=True)

In [7]:
all_queries = set(df_train['query']).union(set(df_val['query'])).union(set(df_test['query']))

In [10]:
set(df_exist['query']) - all_queries

set()

In [12]:
len(set(df_exist['query'])), len(all_queries)

(44197, 123653)

In [16]:
recs_leftover = []
with open('appen_tolabel_query_attr_leftover.json', 'r') as f:
    for l in tqdm(f):
        d = json.loads(l)
        if d['query'] in all_queries:
            recs_leftover.append(d)

455807it [06:31, 1162.92it/s]


In [17]:
df_leftover = pd.DataFrame(recs_leftover)

In [23]:
df_embs = pd.concat([
    df_exist[['query', 'openai_embedding']],
    df_leftover[['query', 'openai_embedding']]
]).drop_duplicates('query')

In [28]:
len(df_embs) / len(all_queries)

0.9998786927935432

In [30]:
df_train_emb = df_train.merge(df_embs, on='query', how='left')
df_val_emb = df_val.merge(df_embs, on='query', how='left')
df_test_emb = df_test.merge(df_embs, on='query', how='left')

In [39]:
df_train_emb.loc[df_train_emb.openai_embedding.isna(), 'openai_embedding'] = df_train_emb.loc[df_train_emb.openai_embedding.isna(), 'query'].apply( 
    lambda x: get_embedding(x, engine=engine))
df_val_emb.loc[df_val_emb.openai_embedding.isna(), 'openai_embedding'] = df_val_emb.loc[df_val_emb.openai_embedding.isna(), 'query'].apply( 
    lambda x: get_embedding(x, engine=engine))
df_test_emb.loc[df_test_emb.openai_embedding.isna(), 'openai_embedding'] = df_test_emb.loc[df_test_emb.openai_embedding.isna(), 'query'].apply( 
    lambda x: get_embedding(x, engine=engine))

In [40]:
df_train_emb[df_train_emb.openai_embedding.isna()], df_val_emb[df_val_emb.openai_embedding.isna()], df_test_emb[df_test_emb.openai_embedding.isna()]

(Empty DataFrame
 Columns: [label_ordering, sample_method, query, category, Rater_Answer, attr_name_value_pairs_normalized, attr_name_value_pairs_custom, attr_name_value_pairs_normalized_text, openai_embedding]
 Index: [],
 Empty DataFrame
 Columns: [label_ordering, sample_method, query, category, Rater_Answer, attr_name_value_pairs_normalized, attr_name_value_pairs_custom, attr_name_value_pairs_normalized_text, openai_embedding]
 Index: [],
 Empty DataFrame
 Columns: [label_ordering, sample_method, query, category, Rater_Answer, attr_name_value_pairs_normalized, attr_name_value_pairs_custom, attr_name_value_pairs_normalized_text, openai_embedding]
 Index: [])

In [41]:
df_train_emb.to_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_query_attr_train_oaiemb.json', 
    lines=True, orient='records')
df_val_emb.to_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_query_attr_val_oaiemb.json', 
    lines=True, orient='records')
df_test_emb.to_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_query_attr_test_oaiemb.json', 
    lines=True, orient='records')

# train simple model

In [42]:
df_train_emb = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_query_attr_train_oaiemb.json', 
    lines=True)
df_val_emb = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_query_attr_val_oaiemb.json', 
    lines=True)
df_test_emb = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_query_attr_test_oaiemb.json', 
    lines=True)